In [10]:
# 연재 중 웹툰 크롤링

from bs4 import BeautifulSoup
from pprint import pprint
import requests
from selenium import webdriver
import pymysql
import re
from urllib.parse import urlparse, parse_qs

# mysql db connect
db = pymysql.connect(host='52.78.23.232', user='root', password='readers7', port=3306, db='readers', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
# browser = webdriver.Chrome('C:/Users/KSY/Documents/GitHub/Readers/crawling/chromedriver.exe')
cursor1 = db.cursor()
cursor2 = db.cursor()
cursor3 = db.cursor()
cursor4 = db.cursor()

#웹 페이지를 열고 소스코드를 읽어오는 작업
def readHtml(url):
    # print(url)
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    html.close()
    return soup


In [98]:
# 연재 중 웹툰 크롤링

sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"
weekday_sql = "insert ignore into toon_weekday(toon_id, toon_weekday)\
               values (%s, %s)"

soup = readHtml("http://comic.naver.com/webtoon/weekday.nhn")

#요일별 웹툰영역 추출하기
data1_list=soup.findAll('div',{'class':'thumb'})
# pprint(data1_list)

#전체 웹툰 리스트
a_list = []
for data1 in data1_list:
    a_list.extend(data1.findAll('a'))

# 모든 요일 링크 추출
for a in a_list:
    src = a['href']
    
    url = urlparse(src)
    toon_id = 'naver_' + str(parse_qs(url.query)['titleId']).replace("['", '').replace("']", '') # ID 생성
    
    toon_url = 'https://comic.naver.com' + src + '&page=' # 웹툰 페이지 추출
    
    soup = readHtml(toon_url+'1') # 네이버 웹툰 첫페이지
    
    # 요일 추출
    if 'weekday=mon' in toon_url:
        weekday = 'mon'
    if 'weekday=tue' in toon_url:
        weekday = 'tue'
    if 'weekday=wed' in toon_url:
        weekday = 'wed'
    if 'weekday=thu' in toon_url:
        weekday = 'thu'
    if 'weekday=fri' in toon_url:
        weekday = 'fri'
    if 'weekday=sat' in toon_url:
        weekday = 'sat'
    if 'weekday=sun' in toon_url:
        weekday = 'sun'
        
    cursor1.execute(weekday_sql, (toon_id, weekday))
    
    detail = soup.find('div', {'class':'detail'})
    toon_name = detail.find('h2')
    
    wrt_nm = str(toon_name.find('span', {'class':'wrt_nm'}))
    wrt_name = re.sub('<.+?>', '', wrt_nm, 0).strip() # 작가 추출
    
    toon_name = str(toon_name).replace('\t', '').replace('컷툰', '')
    toon_name = re.sub('<.+?>', '', toon_name, 0).strip()
    terminator = toon_name.index('\n')
    toon_name = toon_name[:terminator].strip('컷') # 제목 추출
    
    # 장르 추출
    genres = str(detail.find('span', {'class':'genre'}))
    genres = re.sub('<.+?>', '', genres, 0).strip()
    genre_list = genres.split(', ')

    desc = str(detail.find('p')).replace('<br/>', '\n')
    toon_desc = re.sub('<.+?>', '', desc, 0).strip() # 설명 추출
    
    toon_thumb_url = soup.find('img')['src'] # 썸네일 주소 추출
    
    # toon_info
    cursor2.execute(sel_sql)
    result = cursor2.fetchall()
    if (toon_id not in result):
        cursor3.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'naver', wrt_name, 'X', toon_thumb_url))
        for genre_name in genre_list:
            print(genre_name)
            cursor4.execute(genre_sql, (toon_id, genre_name))
    db.commit()

db.close()

InterfaceError: (0, '')

In [26]:
sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"

soup = readHtml("https://comic.naver.com/webtoon/finish.nhn")

#요일별 웹툰영역 추출하기
data1_list=soup.findAll('div',{'class':'thumb'})
# pprint(data1_list)

#전체 웹툰 리스트
a_list = []
for data1 in data1_list:
    a_list.extend(data1.findAll('a'))
# pprint(a_list)

# 웹툰 정보 추출
for a in a_list:
    src = a['href']
    url = 'https://comic.naver.com' + src + '&page=' # 웹툰 페이지 추출
    
    url = urlparse(src)
    toon_id = 'naver_' + str(parse_qs(url.query)['titleId']).replace("['", '').replace("']", '') # ID 생성

    toon_url = 'https://comic.naver.com' + src + '&page=' # 웹툰 페이지 추출
    
    soup = readHtml(toon_url+'1') # 네이버 웹툰 첫페이지
    
    detail = soup.find('div', {'class':'detail'})
    toon_name = detail.find('h2')
    
    wrt_nm = str(toon_name.find('span', {'class':'wrt_nm'}))
    wrt_name = re.sub('<.+?>', '', wrt_nm, 0).strip() # 작가 추출
    
    toon_name = str(toon_name).replace('\t', '').replace('컷툰', '')
    toon_name = re.sub('<.+?>', '', toon_name, 0).strip()
    terminator = toon_name.index('\n')
    toon_name = toon_name[:terminator].strip('컷') # 제목 추출
    
    # 장르 추출
    genres = str(detail.find('span', {'class':'genre'}))
    genres = re.sub('<.+?>', '', genres, 0).strip()
    genre_list = genres.split(', ')

    desc = str(detail.find('p')).replace('<br/>', '\n')
    toon_desc = re.sub('<.+?>', '', desc, 0).strip() # 설명 추출
    
    toon_thumb_url = soup.find('img')['src'] # 썸네일 주소 추출
    
    # toon_info
    cursor1.execute(sel_sql)
    result = cursor1.fetchall()
    if (toon_id not in result):
        cursor2.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'naver', wrt_name, 'O', toon_thumb_url))
        for genre_name in genre_list:
            print(genre_name)
            cursor3.execute(genre_sql, (toon_id, genre_name))
    db.commit()
    
db.close()

에피소드
일상
개그
스토리
일상
드라마
로맨스
스토리
일상
로맨스
에피소드
일상
개그
스토리
판타지
드라마
로맨스
감성
스토리
개그
판타지
드라마
로맨스
에피소드
일상
개그
판타지
드라마
로맨스
스토리
로맨스
스토리
드라마
감성
에피소드
일상
드라마
감성
에피소드
일상
드라마
스토리
개그
로맨스
스토리
드라마
옴니버스
일상
개그
옴니버스
스릴러
스토리
판타지
스토리
드라마
스토리
스릴러
스토리
개그
드라마
스포츠
스토리
스릴러
스토리
스토리
판타지
액션
스토리
옴니버스
개그
스토리
일상
개그
판타지
스토리
드라마
감성
스토리
일상
판타지
드라마
감성
스토리
개그
판타지
에피소드
드라마
감성
스토리


KeyboardInterrupt: 

In [ ]:
### 네이버 웹툰 회차정보 크롤링 ###

sel_url_sql = 'select toon_id, toon_url from toon_info where toon_site = "naver"'
sel_dup_sql = 'select toon_id, epi_name from epi_info where toon_id = %s and epi_name = %s'
ins_epi_info_sql = 'insert into epi_info(toon_id, epi_name, epi_url, epi_thumb_url, epi_date)\
                    values (%s, %s, %s, %s, %s)'

cursor1.execute(sel_url_sql)
result = cursor1.fetchall()
   
for row in result:
    toon_id = row['toon_id']
    toon_url = row['toon_url']
    
    num = 1 # 페이지 탐색용
    have_to_break = 0 # 루프 탈출용
    
    while 1:
        real_url = toon_url+str(num) # 실제 웹툰 링크, 첫 페이지부터
        num = num + 1 # 다음 페이지 탐색을 위해
        soup = readHtml(real_url)

        webtoon_area = soup.find("table", {"class": "viewList"}).findAll('tr')
        
        if (have_to_break == 1):
            break
        
        for index in webtoon_area:
            info = index.find('a')
            if (index.find('a')):
                if (index.find('td', {'class' : 'num'})):
                    info = index.find('a')
                    epi_url = "https://comic.naver.com" + info['href'] # 에피소드 링크 추출
                    img = info.find('img')
                    epi_thumb_url = img['src'] # 썸네일 링크 추출
                    epi_name = img['title'] # 에피소드 제목 추출
                    date = index.find('td', {'class' : 'num'}).get_text()
                    epi_date = date.replace(".", "-") # 업데이트 날짜 추출
                    
                    cursor2.execute(sel_dup_sql, (toon_id, epi_name))
                    if cursor2.fetchall():
                        have_to_break = 1
                        break
    
                    else:
                        cursor3.execute(ins_epi_info_sql, (toon_id, epi_name, epi_url, epi_thumb_url, epi_date))
                        db.commit()
                
            else:
                continue

db.close()